In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from os import listdir
from os.path import isfile, join, isdir


Load data


In [22]:
import chardet
RELATIVE_DATA_PATH = './data'
countries = [f for f in listdir(RELATIVE_DATA_PATH) if isdir(join(RELATIVE_DATA_PATH, f))]

frames = []
for country in countries:
    print(f"Parsing {country}")
    pollutantsPath = join(RELATIVE_DATA_PATH, country)
    pollutants = [f for f in listdir(pollutantsPath) if isdir(join(pollutantsPath, f)) ]
    
    for pollutant in pollutants:
        pollutantPath = join(pollutantsPath, pollutant)
        dataCSVs = [f for f in listdir(pollutantPath)]
        
        for dataCSV in dataCSVs:
            dataCSVPath = join(pollutantPath, dataCSV)
            
            with open(dataCSVPath, 'rb') as f:
                info = chardet.detect(f.read())
                currentCSV = pd.read_csv(dataCSVPath, sep=',', encoding=info.get('encoding'))
                frames.append(currentCSV)
            
data = pd.concat(frames)
metadata = pd.read_csv(join(RELATIVE_DATA_PATH, 'metadata.csv'))


Parsing CH


/Users/so/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Data


In [23]:
data.sample(5)


,AirPollutant,AirPollutantCode,AirQualityNetwork,AirQualityStation,AirQualityStationEoICode,AveragingTime,Concentration,Countrycode,DatetimeBegin,DatetimeEnd,Namespace,Sample,SamplingPoint,SamplingProcess,UnitOfMeasurement,Unnamed: 0,Validity,Verification
5040,CO,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,NET-CH003A,STA-CH0002R,CH0002R,hour,0.226,CH,2013-03-12 16:00:00 +01:00,2013-03-12 17:00:00 +01:00,CH.BAFU.AQ,SPO_F-CH0002R_00010_100_100,SPO-CH0002R_00010_100,SPP-CH_A_NDIR_horibaAPMA360,mg/m3,NaN,1.0,1.0
15238,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,NET-CH003A,STA-CH0002R,CH0002R,hour,4.600,CH,2013-07-12 17:00:00 +01:00,2013-07-12 18:00:00 +01:00,CH.BAFU.AQ,SPO_F-CH0002R_00008_100_102,SPO-CH0002R_00008_100,SPP-CH_A_5150551505,µg/m3,NaN,1.0,1.0
14657,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,NET-CH003A,STA-CH0002R,CH0002R,hour,25.700,CH,2013-11-28 15:00:00 +01:00,2013-11-28 16:00:00 +01:00,CH.BAFU.AQ,SPO_F-CH0002R_00008_100_102,SPO-CH0002R_00008_100,SPP-CH_A_5150551505,µg/m3,NaN,1.0,1.0
5020,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,NET-CH003A,STA-CH0003R,CH0003R,hour,4.400,CH,2013-03-09 05:00:00 +01:00,2013-03-09 06:00:00 +01:00,CH.BAFU.AQ,SPO_F-CH0003R_00008_100_102,SPO-CH0003R_00008_100,SPP-CH_A_chemi_horibaAPNA370,µg/m3,NaN,1.0,1.0
11138,SO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,NET-CH003A,STA-CH0005A,CH0005A,hour,0.700,CH,2013-06-05 11:00:00 +01:00,2013-06-05 12:00:00 +01:00,CH.BAFU.AQ,SPO_F-CH0005A_00001_502_502,SPO-CH0005A_00001_502,SPP-CH_A_5150551505,µg/m3,NaN,1.0,1.0


Metadata


In [26]:
metadata.sample(5)


,Countrycode,Namespace,AirQualityNetwork,AirQualityStation,AirQualityStationNatCode,AirQualityStationEoICode,AirQualityStationArea,SamplingPoint,SamplingProcess,Sample,...,MeasurementEquipment,MeasurementType,MeasurementMethod,AirPollutantCode,AirPollutant,AirQualityStationType,Projection,Longitude,Latitude,Altitude
26182,FR,FR.LCSQA-INERIS.AQ,NET-FR074A,STA-FR06009,FR06009,FR06009,urban,SPO-FR06009_00005_100,SPP-FR_A_TEOM_09000,SPO_F-FR06009_00005_100_100,...,NaN,automatic,http://dd.eionet.europa.eu/vocabulary/aq/measu...,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,PM10,background,EPSG:4979,3.234167,50.175278,59.0
21132,ES,ES.BDCA.AQD,NET_ES205A,STA_ES1759A,38038012,ES1759A,urban,SP_38038012_10_46,SPP_38038012_10_46.1,SAM_38038012_10_46,...,http://dd.eionet.europa.eu/vocabulary/aq/measu...,automatic,http://dd.eionet.europa.eu/vocabulary/aq/measu...,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,PM10,industrial,EPSG:4979,-16.277692,28.451022,183.0
13123,DE,http://gdi.uba.de/arcgis/rest/services/inspire...,NET.DE_HE,STA.DE_DEHE094,DEHE094,DEHE094,urban,SPO.DE_DEHE094_NOx_dataGroup1,SPP.DE_DEHE094_NOx_automatic_chemi_Duration-30...,SAM.DE_DEHE094_1,...,NaN,automatic,http://dd.eionet.europa.eu/vocabulary/aq/measu...,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,NOX as NO2,traffic,EPSG:4979,8.832221,49.827775,161.0
15542,DE,http://gdi.uba.de/arcgis/rest/services/inspire...,NET.DE_SN,STA.DE_DESN083,DESN083,DESN083,urban,SPO.DE_DESN083_NOx_dataGroup1,SPP.DE_DESN083_NOx_automatic_chemi_Duration-1hour,SAM.DE_DESN083_1,...,NaN,automatic,http://dd.eionet.europa.eu/vocabulary/aq/measu...,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,NOX as NO2,traffic,EPSG:4979,12.903367,50.845740,327.0
38076,IT,IT.ISPRA.AQD,NET.IT255A,STA.IT0702A,803513,IT0702A,urban,SPO.IT0702A_10_NDIR_1989-04-01_00:00:00,SPP.IT0702A_10_NDIR_1989-04-01_00:00:00,SAM.IT0702A_10_NDIR_1989-04-01_00:00:00,...,http://dd.eionet.europa.eu/vocabulary/aq/measu...,automatic,http://dd.eionet.europa.eu/vocabulary/aq/measu...,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,CO,traffic,EPSG:4979,10.624444,44.699722,50.0


In [15]:
# Print shape of data
data.shape
            
        
    


(140043, 18)